# Pytorch Training UI

### Import Libraries

In [1]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

In [2]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    py_file_location = "./drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch" # my private packages are stored here
    home_directory = './drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/' # my home directory is stored in ./LAB of google drive
    !pip install einops
else:
    py_file_location = './PrivatePackages/pytorch'
    home_directory = './'

sys.path.append(os.path.abspath(py_file_location))

from environment import *
from utils import *

from sklearn.model_selection import train_test_split

In [3]:
from model.model_class import LSTM, BERT, LSTM_DANN, BERT_DANN

### Set Seed and Load Data

In [4]:
SEED = 2608

In [5]:
data1 = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open(home_directory + './data/raw/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

# create domain labels for data
for i in range(len(data1)):
    data1[i]['domain'] = 1
for i in range(len(data2)):
    data2[i]['domain'] = 2

In [6]:
# Train Val Test Split

# get labels for stratification
label1 = [instance['label'] for instance in data1]
label2 = [instance['label'] for instance in data2]

train_ix_1, val_test_ix_1 = train_test_split(range(len(data1)), test_size=0.3, random_state=SEED, stratify = label1)
train_ix_2, val_test_ix_2 = train_test_split(range(len(data2)), test_size=0.3, random_state=SEED, stratify = label2)
val_ix_1, test_ix_1 = train_test_split(val_test_ix_1, test_size=0.5, random_state=SEED, stratify = [data1[i]['label'] for i in val_test_ix_1])
val_ix_2, test_ix_2 = train_test_split(val_test_ix_2, test_size=0.5, random_state=SEED, stratify = [data2[i]['label'] for i in val_test_ix_2])

# split data according to the index from train_test_split
train_data_1 = [data1[i] for i in train_ix_1]
val_data_1 = [data1[i] for i in val_ix_1]
test_data_1 = [data1[i] for i in test_ix_1]
train_data_2 = [data2[i] for i in train_ix_2]
val_data_2 = [data2[i] for i in val_ix_2]
test_data_2 = [data2[i] for i in test_ix_2]

# combine the data
train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2
test_data = test_data_1 + test_data_2

---
Preprocess data

In [7]:
def W2V_DataFactory(data: list, context_window: int, seed: int, raw_token_pytorch_map: dict, k) -> list:

    """ Get W2V training data """
    
    assert context_window % 2 == 1, 'context window must be odd'

    np.random.seed(seed)

    MAX_SAMPLED_NEGATIVE_TOKENS = 10000

    retokenised_keys = list(raw_token_pytorch_map.keys())

    negative_tokens = np.random.choice(retokenised_keys, MAX_SAMPLED_NEGATIVE_TOKENS)

    negative_up_to = 0

    w2v_data = []

    for instance in tqdm(data):
        tokens = [context_window//2 * 'CLS'] + instance['text'] + [context_window//2 * raw_token_pytorch_map['PAD']]

        for i in range(context_window//2, len(tokens) - context_window//2):
            
            focus_token_retokenised = raw_token_pytorch_map.get(tokens[i], raw_token_pytorch_map['UNK'])
            context_words = set()

            for j in range(-context_window//2, context_window//2+1):
                if j != 0: # don't want to make positive sample with self
                    if tokens[j] in context_words: # CLS and Padding (being start and end) being repeated
                        continue 
                    
                    new_instance = {'token': focus_token_retokenised, 'context': raw_token_pytorch_map.get(tokens[j], raw_token_pytorch_map['UNK']), 'label': 1}
                    w2v_data.append(new_instance)
                    context_words.add(tokens[j])
            
            for j in range(len(context_words)): # sample the same number of negatives
                # TODO: different for each round?
                while True:
                    
                    if negative_up_to == MAX_SAMPLED_NEGATIVE_TOKENS:
                        negative_up_to = 0
                        #TODO: shuffle

                    sampled_negative_retokenised = negative_tokens[negative_up_to]
                    negative_up_to += 1
                    if sampled_negative_retokenised not in context_words: # didn't sample a positive case
                        break

                new_instance = {'token': focus_token_retokenised, 'context': sampled_negative_retokenised, 'label': 0}
                w2v_data.append(new_instance)
    
    return w2v_data

In [8]:
MAX_SENTENCE_LENGTH = 128
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right

In [9]:
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance =MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance =MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)

100%|██████████| 2700/2700 [00:00<00:00, 142344.21it/s]


In [10]:
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 

100%|██████████| 12600/12600 [00:00<00:00, 65548.11it/s]


In [11]:
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    MAX_SENTENCE_LENGTH, \
                                                                        raw_token_pytorch_map, \
                                                                            CLS=True)

100%|██████████| 2700/2700 [00:00<00:00, 75206.17it/s]


In [12]:
train_x = train_x[:200]
train_y = train_y[:200]
train_dom = train_dom[:200]
val_x = val_x[:200]
val_y = val_y[:200]
val_dom = val_dom[:200]

In [13]:
pos_prior, neg_prior = get_distribution(train_y)

In [14]:
train_w2v_data = W2V_DataFactory(train_data, context_window= W2V_CONTEXT_WINDOW, seed = SEED, raw_token_pytorch_map = raw_token_pytorch_map)
val_w2v_data = W2V_DataFactory(val_data, context_window= W2V_CONTEXT_WINDOW, seed = SEED, raw_token_pytorch_map = raw_token_pytorch_map)

TypeError: W2V_DataFactory() missing 1 required positional argument: 'k'

In [ ]:
# class Dataset():
#     """ Pytorch style dataset """

#     def __init__(self, data, maxlen):
#         self.data = data
#         self.maxlen = maxlen
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         return self.data[index]['text'], self.data[index]['label']
#         # return self.data[index]['text'], self.data[index]['label'], self.data[index]['domain']

---
### Models

#### 1. Prediction only

In [ ]:
# LSTM

class LSTM_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 1
    bidirectional = True
    n_heads = 8
    dropout = 0.1
    n_mlp_layers = 0
    flatten = False
    activation = nn.ReLU()
    res_learning = False
    mask_flag = False # TODO
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 5
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    regularisation_loss = None
    scheduler = True
    grad_clip = True
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_Classifier'
    


model = LSTM(LSTM_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, val_x, val_y)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/99 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:101: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
  3%|▎       

KeyboardInterrupt: 

In [ ]:
# BERT

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 128
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3 
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 0
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 2
    lr = 1e-3
    patience = 2
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'
    


model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, val_x, val_y)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, best_epoch, evaluation_mode = True)

 99%|█████████▉| 98/99 [03:31<00:02,  2.99s/it]

---
# DANN

In [ ]:
# LSTM_DANN

class LSTM_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 1
    bidirectional = True
    n_heads = 0
    dropout = 0.1
    n_mlp_clf_layers = 0
    n_mlp_dom_layers = 0
    flatten = False
    activation = nn.ReLU()
    res_learning = False
    mask_flag = False # TODO
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 5
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    alpha = 0.5
    regularisation_loss = None
    scheduler = True
    grad_clip = True
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_DANN_Classifier'
    


model = LSTM_DANN(LSTM_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|█████████

Epoch 1 Train | Classification Loss:  0.3456 | Accuracy:  0.4900| F1:  0.6047 | Balanced Accuracy:  0.4743 | Domain Loss:  0.3872 | Domain Accuracy:  0.4900


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1 Val   | Classification Loss:  0.3059 | Accuracy:  0.9400| F1:  0.9691 | Balanced Accuracy:  0.9400 | Domain Loss:  1.0035 | Domain Accuracy:  0.0600


  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


...

  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


Epoch 7 Train | Classification Loss:  0.3306 | Accuracy:  0.6600| F1:  0.7500 | Balanced Accuracy:  0.6407 | Domain Loss:  0.2772 | Domain Accuracy:  0.6600


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 7 Val   | Classification Loss:  0.3336 | Accuracy:  0.7350| F1:  0.8473 | Balanced Accuracy:  0.7350 | Domain Loss:  0.4540 | Domain Accuracy:  1.0000

Epoch 5 Val   | Classification Loss:  0.3042 | Accuracy:  0.9450| F1:  0.9717 | Balanced Accuracy:  0.9450 | Domain Loss:  0.7634 | Domain Accuracy:  0.0250


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [17]:
# BERT

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 128
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3 
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 0
    n_mlp_dom_layers = 0
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 32
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    alpha = 0.5
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'
    


model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

100%|██████████| 2/2 [00:03<00:00,  1.57s/it]


Epoch 1 Train | Classification Loss:  0.4664 | Accuracy:  0.5100| F1:  0.4494 | Balanced Accuracy:  0.5185 | Domain Loss:  0.8735 | Domain Accuracy:  0.5100


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Epoch 1 Val   | Classification Loss:  0.1735 | Accuracy:  1.0000| F1:  1.0000 | Balanced Accuracy:  1.0000 | Domain Loss:  6.3180 | Domain Accuracy:  0.0000


100%|██████████| 2/2 [00:04<00:00,  2.07s/it]


...

100%|██████████| 2/2 [00:02<00:00,  1.48s/it]


Epoch 32 Train | Classification Loss:  0.3482 | Accuracy:  0.5500| F1:  0.6691 | Balanced Accuracy:  0.5303 | Domain Loss:  0.6592 | Domain Accuracy:  0.5500


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 32 Val   | Classification Loss:  0.3042 | Accuracy:  0.8450| F1:  0.9160 | Balanced Accuracy:  0.8450 | Domain Loss:  1.2318 | Domain Accuracy:  0.0000

Epoch 1 Val   | Classification Loss:  0.1735 | Accuracy:  1.0000| F1:  1.0000 | Balanced Accuracy:  1.0000 | Domain Loss:  6.3180 | Domain Accuracy:  0.0000


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
